In [1]:
import csv
import numpy as np
from ultralytics import YOLO
import cv2
import os
import pandas as pd
import tqdm

In [2]:
detect_model = YOLO('yolov8x.pt')

In [3]:
df = pd.read_csv('raw.csv')

In [19]:
N = 10_000 # number of frames to use for the dataset
subjects = df['S'].unique()
frames_per_subject = N // len(subjects)

cols = df.columns.tolist()


cols.insert(6, 'box_x')
cols.insert(7, 'box_y')
cols.insert(8, 'box_w')
cols.insert(9, 'box_h')

df_small = pd.DataFrame(columns=cols)

for s in tqdm.tqdm(subjects):
    df_sub = df[df['S'] == s]
    
    # get random frames
    indexes = np.random.choice(df_sub.index, frames_per_subject, replace=False)
    
    df_small = pd.concat([df_small, df_sub.loc[indexes]])

  0%|          | 0/13 [00:00<?, ?it/s]C:\Users\rynha\AppData\Local\Temp\ipykernel_9168\2494914295.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_small = pd.concat([df_small, df_sub.loc[indexes]])
100%|██████████| 13/13 [00:00<00:00, 28.51it/s]


In [24]:
df_detect = pd.DataFrame(columns=cols)

root_path = 'E:\data_processed'
for index, row in tqdm.tqdm(df_small.iterrows(), total=len(df_small)):
    img_path = row['img_path']
    img_path = os.path.join(root_path, img_path)
    img = cv2.imread(img_path)

    results = detect_model(img,classes=[0],verbose=False)
    
    boxes = results[0].boxes
    if len(boxes.xywh) == 0:
        continue
    xywh = boxes.xywh.numpy()[0]
    cx, cy, w, h = xywh
    xywh = [cx-w/2, cy-h/2, w, h]

    # add row to df_detect
    new_row_df = pd.DataFrame([row])
    
    new_row_df['box_x'] = round(xywh[0], 2)
    new_row_df['box_y'] = round(xywh[1], 2)
    new_row_df['box_w'] = round(xywh[2], 2)
    new_row_df['box_h'] = round(xywh[3], 2)

    df_detect = pd.concat([df_detect, new_row_df], ignore_index=True)

#     im_show = img.copy()

#     im_show = cv2.rectangle(im_show, (int(xywh[0]), int(xywh[1])), (int(xywh[0]+xywh[2]), int(xywh[1]+xywh[3])), (0, 255, 0), 2)
#     im_show = cv2.resize(im_show, (288, 450))
#     cv2.imshow('img', im_show)
#     key = cv2.waitKey(1)
#     if key == 27:
#         break

# cv2.destroyAllWindows()

df_detect.to_csv('detect.csv', index=False)

  0%|          | 0/9997 [00:00<?, ?it/s]C:\Users\rynha\AppData\Local\Temp\ipykernel_9168\1316238239.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_detect = pd.concat([df_detect, new_row_df], ignore_index=True)
100%|██████████| 9997/9997 [1:22:04<00:00,  2.03it/s]


In [27]:
# Load image
img = cv2.imread('E:/data_processed/frames/C5/S1/C5S1A7D2/C5S1A7D2_0518.jpg')

# Detect bounding box
results = detect_model(img, classes=[0], verbose=False)

# Get center (cx, cy), width (w), and height (h) of the bounding box
xyxy = results[0].boxes.xyxy.numpy()[0]

x1, y1, x2, y2 = xyxy[0], xyxy[1], xyxy[2], xyxy[3]

# Copy the image for drawing
im_show = img.copy()

# Draw the rectangle
im_show = cv2.rectangle(im_show, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

im_show = cv2.resize(im_show, (288, 450))

# Display the image
cv2.imshow('img', im_show)
cv2.waitKey(0)
cv2.destroyAllWindows()